# Essence Wars: Quickstart Tutorial

Welcome to **Essence Wars**, a high-performance card game environment designed for reinforcement learning research!

This notebook will get you up and running in 5 minutes. You'll learn how to:
1. Install the package
2. Create and play a game
3. Understand the state representation
4. Use built-in AI agents

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/christianwissmann85/ai-cardgame/blob/master/notebooks/01_quickstart.ipynb)

## 1. Installation & Setup

**Important:** The game requires data files (cards, decks) from the repo root. Run this cell first!

In [1]:
# Setup: Change to repo root directory (required for data files)
import os
from pathlib import Path


# Find repo root (contains 'data' folder)
def find_repo_root():
    path = Path.cwd()
    while path != path.parent:
        if (path / 'data' / 'cards').exists():
            return path
        path = path.parent
    return None

repo_root = find_repo_root()
if repo_root:
    os.chdir(repo_root)
    print(f"Working directory: {os.getcwd()}")
else:
    print("Warning: Could not find repo root. Make sure you're running from within the ai-cardgame repo.")

Working directory: /home/chris/ai-cardgame


In [2]:
# Uncomment and run this cell in Google Colab
# import subprocess
# import sys
#
# # Check if we're in Colab
# IN_COLAB = 'google.colab' in sys.modules
#
# if IN_COLAB:
#     print("Installing Rust toolchain...")
#     !curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y
#     import os
#     os.environ['PATH'] = f"{os.environ['HOME']}/.cargo/bin:{os.environ['PATH']}"
#
#     print("\nCloning repository...")
#     !git clone https://github.com/christianwissmann85/ai-cardgame.git
#     os.chdir('ai-cardgame')
#
#     print("\nInstalling essence-wars...")
#     !pip install -e .
#     print("\nInstallation complete!")

## 2. Your First Game

Let's create a game and see what it looks like!

In [3]:
from essence_wars._core import PyGame

# Create a new game with two decks
game = PyGame(
    deck1="artificer_tokens",    # Player 1's deck
    deck2="broodmother_swarm",   # Player 2's deck
    game_mode="attrition"         # Win by reducing enemy life to 0
)

# Reset with a random seed (for reproducibility)
game.reset(seed=42)

print("Game created!")
print(f"Turn: {game.turn_number()}")
print(f"Current player: {game.current_player()}")
print(f"Game over: {game.is_done()}")

Game created!
Turn: 1
Current player: 0
Game over: False


### Visualizing the Game State

We provide an ASCII renderer to visualize the board state:

In [4]:
from essence_wars.viz import GameRenderer

# Create a renderer
renderer = GameRenderer()

# Print the current game state
renderer.print(game)

=== Essence Wars: Turn 1 ===

Player 2

Life: 20  |  Essence:  0  |  AP: 0  |  Hand: 6  |  Deck: 19

Board: [    ]  [    ]  [    ]  [    ]  [    ]

────────────────────────────────────────────────────────────

Player 1 [*]

Life: 20  |  Essence:  1  |  AP: 3  |  Hand: 5  |  Deck: 25

Board: [    ]  [    ]  [    ]  [    ]  [    ]

Legal actions: 1

### Understanding the Display

```
=== Essence Wars: Turn 1 ===

Player 2                          <- Opponent
  Life: 20  |  Essence: 0  |  AP: 0  |  Hand: 6  |  Deck: 19
  Board: [    ]  [    ]  [    ]  [    ]  [    ]   <- 5 creature slots

────────────────────────────────────────────────

Player 1 [*]                      <- You (active player marked with *)
  Life: 20  |  Essence: 1  |  AP: 3  |  Hand: 5  |  Deck: 25
  Board: [    ]  [    ]  [    ]  [    ]  [    ]
```

**Key stats:**
- **Life**: Reduce enemy to 0 to win
- **Essence**: Mana for playing cards (grows each turn, max 10)
- **AP**: Action Points (3 per turn)
- **Hand/Deck**: Cards available

**Creature display:**
- `[A/H]` = Attack/Health
- `z` = exhausted (can't attack)
- `!` = can attack
- Keywords: `G`=Guard, `R`=Rush, `L`=Lethal, etc.

## 3. Playing the Game

Let's play some turns using the built-in GreedyBot!

In [5]:
# Reset for a fresh game
game.reset(seed=123)

print("Initial state:")
renderer.print(game)

# Play 10 actions using the greedy bot
print("\n" + "="*50)
print("Playing 10 actions with GreedyBot...")
print("="*50 + "\n")

for i in range(10):
    if game.is_done():
        print("Game ended!")
        break

    # Get action from greedy bot
    action = game.greedy_action()

    # Apply the action
    reward, done = game.step(action)

    print(f"Action {i+1}: index={action}, reward={reward}, done={done}")

print("\nFinal state:")
renderer.print(game)

Initial state:


=== Essence Wars: Turn 1 ===

Player 2

Life: 20  |  Essence:  0  |  AP: 0  |  Hand: 6  |  Deck: 19

Board: [    ]  [    ]  [    ]  [    ]  [    ]

...

────────────────────────────────────────────────────────────

Player 1 [*]

Life: 18  |  Essence:  0  |  AP: 2  |  Hand: 5  |  Deck: 23

Board: [    ]  [    ]  [    ]  [    ]  [2/4]

Legal actions: 1

## 4. Understanding the State Tensor

For machine learning, the game state is represented as a **326-dimensional float tensor**.

This is what your neural network sees!

In [6]:
import numpy as np

# Get the state tensor
obs = np.array(game.observe())

print(f"State tensor shape: {obs.shape}")
print(f"State tensor dtype: {obs.dtype}")
print("\nFirst 20 values:")
print(obs[:20])

State tensor shape: (326,)
State tensor dtype: float32

First 20 values:
[ 1.6666667e-01  0.0000000e+00  0.0000000e+00 -1.0000000e+00
  0.0000000e+00  0.0000000e+00  8.9999998e-01  0.0000000e+00
  6.6666669e-01  7.6666665e-01  5.0000000e-01  1.0630000e+03
  1.0560000e+03  1.0680000e+03  1.0170000e+03  1.0610000e+03
  0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]


### Tensor Layout

The 326 floats encode:

| Section | Size | Description |
|---------|------|-------------|
| Global | 6 | Turn, current player, game over, winner |
| Player 1 | 75 | Life, essence, AP, hand, creatures, supports |
| Player 2 | 75 | Same structure as Player 1 |
| Card IDs | 170 | Card identifiers for embedding lookup |

All values are **normalized to [0, 1]** for neural network training.

In [7]:
# Let's decode some values manually
game.reset(seed=42)
obs = np.array(game.observe())

print("=== Decoded Global State ===")
print(f"Turn (normalized):     {obs[0]:.3f}  -> Turn {int(obs[0] * 30)}")
print(f"Current player:        {obs[1]:.0f}")
print(f"Game over:             {obs[2]:.0f}")
print(f"Winner:                {obs[3]:.0f}  (-1 = ongoing)")

print("\n=== Player 1 Base Stats ===")
p1_start = 6
print(f"Life (normalized):     {obs[p1_start]:.3f}  -> {int(obs[p1_start] * 20)} HP")
print(f"Essence (normalized):  {obs[p1_start+1]:.3f}  -> {int(obs[p1_start+1] * 10)} mana")
print(f"Action Points:         {obs[p1_start+2]:.3f}  -> {int(obs[p1_start+2] * 3)} AP")
print(f"Deck size:             {obs[p1_start+3]:.3f}  -> {int(obs[p1_start+3] * 30)} cards")
print(f"Hand size:             {obs[p1_start+4]:.3f}  -> {int(obs[p1_start+4] * 10)} cards")

=== Decoded Global State ===
Turn (normalized):     0.033  -> Turn 1
Current player:        0
Game over:             0
Winner:                -1  (-1 = ongoing)

=== Player 1 Base Stats ===
Life (normalized):     1.000  -> 20 HP
Essence (normalized):  0.100  -> 1 mana
Action Points:         1.000  -> 3 AP
Deck size:             0.833  -> 25 cards
Hand size:             0.500  -> 5 cards


## 5. Action Space and Legal Moves

The game has a **fixed 256-action space** to work with neural networks.

Not all actions are legal at every state - that's where the **action mask** comes in.

In [8]:
# Get the legal action mask
mask = np.array(game.action_mask())

print(f"Action mask shape: {mask.shape}")
print(f"Legal actions: {int(mask.sum())} out of 256")

# Find which actions are legal
legal_indices = np.where(mask > 0.5)[0]
print(f"\nLegal action indices: {legal_indices}")

Action mask shape: (256,)
Legal actions: 1 out of 256

Legal action indices: [255]


### Action Space Layout

| Range | Action Type | Formula |
|-------|-------------|--------|
| 0-99 | Play Card | `hand_idx * 10 + target_slot` |
| 100-149 | Attack | `100 + attacker_slot * 10 + target` |
| 150-249 | Use Ability | `150 + slot * 20 + ability * 10 + target` |
| 255 | End Turn | Always legal |

The mask tells your agent which actions are currently valid.

In [9]:
def decode_action(action_idx: int) -> str:
    """Decode action index to human-readable string."""
    if action_idx == 255:
        return "EndTurn"
    elif action_idx < 100:
        hand_idx = action_idx // 10
        target = action_idx % 10
        return f"PlayCard(hand={hand_idx}, slot={target})"
    elif action_idx < 150:
        idx = action_idx - 100
        attacker = idx // 10
        target = idx % 10
        return f"Attack(slot={attacker}, target={target})"
    else:
        idx = action_idx - 150
        slot = idx // 20
        ability = (idx % 20) // 10
        target = idx % 10
        return f"UseAbility(slot={slot}, ability={ability}, target={target})"

print("Legal actions decoded:")
for idx in legal_indices[:10]:  # Show first 10
    print(f"  {idx}: {decode_action(idx)}")
if len(legal_indices) > 10:
    print(f"  ... and {len(legal_indices) - 10} more")

Legal actions decoded:
  255: EndTurn


## 6. Built-in AI Agents

Essence Wars comes with three built-in agents:

| Agent | Description | Strength |
|-------|-------------|----------|
| **Random** | Picks uniformly from legal actions | Baseline |
| **Greedy** | Evaluates each action, picks best | Medium |
| **MCTS** | Monte Carlo Tree Search | Strong |

These are useful as training opponents and evaluation baselines.

In [10]:
# Compare the agents on the same position
game.reset(seed=42)

# Get actions from each agent
random_action = game.random_action()
greedy_action = game.greedy_action()
mcts_action = game.mcts_action(simulations=50)  # 50 simulations for speed

print("Agent decisions on the same position:")
print(f"  Random: {random_action} -> {decode_action(random_action)}")
print(f"  Greedy: {greedy_action} -> {decode_action(greedy_action)}")
print(f"  MCTS:   {mcts_action} -> {decode_action(mcts_action)}")

Agent decisions on the same position:
  Random: 255 -> EndTurn
  Greedy: 255 -> EndTurn
  MCTS:   255 -> EndTurn


## 7. Play a Full Game

Let's watch two bots play against each other!

In [11]:
# Fresh game
game.reset(seed=999)

turn = 0
while not game.is_done():
    current_player = game.current_player()

    # Player 0 uses MCTS, Player 1 uses Greedy
    if current_player == 0:
        action = game.mcts_action(simulations=20)  # Fast MCTS
        agent_name = "MCTS"
    else:
        action = game.greedy_action()
        agent_name = "Greedy"

    reward, done = game.step(action)
    turn += 1

    # Safety limit
    if turn > 200:
        print("Game too long, breaking...")
        break

print(f"Game finished after {turn} actions!")
print(f"Turn number: {game.turn_number()}")

# Check winner
p1_reward = game.get_reward(0)
p2_reward = game.get_reward(1)

if p1_reward > 0:
    print("Winner: Player 1 (MCTS)")
elif p2_reward > 0:
    print("Winner: Player 2 (Greedy)")
else:
    print("Result: Draw")

print("\nFinal board:")
renderer.print(game)

Game finished after 38 actions!
Turn number: 14
Winner: Player 2 (Greedy)

Final board:


=== Essence Wars: GAME OVER - Player 2 Wins! ===

Player 2 [*]

Life: 20  |  Essence:  7  |  AP: 3  |  Hand: 5  |  Deck: 12

Board: [3/3]z  [3/3]! [R]  [2/2]! [P]  [3/2]! [R]  [2/3]!

...

────────────────────────────────────────────────────────────

Player 1

Life:  0  |  Essence:  7  |  AP: 3  |  Hand: 10  |  Deck: 19

Board: [    ]  [    ]  [    ]  [    ]  [    ]

Legal actions: 0

## 8. Available Decks

Essence Wars has multiple pre-built decks across three factions:

In [12]:
# List all available decks
decks = PyGame.list_decks()

print("Available decks:")
for deck in sorted(decks):
    print(f"  - {deck}")

Available decks:
  - alpha_frenzy
  - architect_fortify
  - archon_burst
  - artificer_tokens
  - broodmother_swarm
  - colossus_wall
  - grove_regenerate
  - kael_assassin
  - plague_volatile
  - shadow_weaver
  - sovereign_lifesteal
  - vex_piercing


### Factions

| Faction | Style | Keywords |
|---------|-------|----------|
| **Argentum Combine** | Defensive, constructs | Guard, Shield, Piercing |
| **Symbiote Circles** | Aggressive, swarm | Rush, Lethal, Regenerate |
| **Obsidion Syndicate** | Burst damage, control | Lifesteal, Stealth, Quick |

## 9. Game Modes

Two game modes are available:

| Mode | Win Condition |
|------|---------------|
| **Attrition** (default) | Reduce enemy life to 0, or have more life at turn 30 |
| **Essence Duel** | First to 50 Victory Points (face damage dealt) |

In [13]:
# Create an Essence Duel game
duel_game = PyGame(
    deck1="artificer_tokens",
    deck2="broodmother_swarm",
    game_mode="essence_duel"  # Victory Points mode
)
duel_game.reset(seed=42)

print("Essence Duel mode created!")
print("In this mode, deal 50 face damage to win.")

Essence Duel mode created!
In this mode, deal 50 face damage to win.


## Next Steps

Congratulations! You've learned the basics of Essence Wars.

**Continue with:**
- **02_environment.ipynb** - Deep dive into the environment API
- **03_dataset_exploration.ipynb** - Explore pre-generated MCTS datasets
- **04_behavioral_cloning.ipynb** - Train your first neural network agent
- **05_alphazero_training.ipynb** - Advanced self-play training

**Resources:**
- [GitHub Repository](https://github.com/christianwissmann85/ai-cardgame)
- [Documentation](https://christianwissmann85.github.io/essence-wars/)
- [Hugging Face Datasets](https://huggingface.co/datasets/christianwissmann85/essence-wars)

Happy training!